In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rc("font", size=14)
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc


In [5]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '6ed9d167-b2e6-41b8-9500-35e6df64d9dc'
resource_group = 'MLRG'
workspace_name = 'erbbimlws'

workspace = Workspace(subscription_id, resource_group, workspace_name)



Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code CQ2EFGDEG to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [6]:
df = Dataset.get_by_name(workspace, name='Time_Deposits_Data_Set')
df.to_pandas_dataframe()

,Customer_ID,Gender,Birth_Date,Ref_Date,Marital_Status,Children_Num,Occupation_Category,Total_Income,Payroll_Flag,Business_Flag,...,Transfer_Trans_Amount,Credit_Cards_Installments,Credit_Cards_Payments_Num,Credit_Cards_Purchases_Num,Credit_Cards_Witrhdrawals_Num,Credit_Cards_Payments_Amount,Credit_Cards_Purchases_Amount,Credit_Cards_Witrhdrawals_Amount,Arrears_Months_Max,Time_Deposits_Flag
0,36163,Male,1968-01-20,2008-12-31,Single,0,Employees,17637,0,0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,1,False
1,5904434,Male,1973-04-07,2008-12-31,Single,0,Rentiers,5800,0,0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,1,False
2,5904453,Male,1960-10-17,2008-12-31,Married,2,Rentiers,14000,0,0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,1,False
3,855752,Male,1969-07-28,2008-12-31,Married,2,Business-Owners,54814,0,0,...,105.000000,1.666667,0.666667,1.0,0.0,141.666666,230.45,0.0,1,False
4,81,Male,1950-07-27,2008-12-31,Married,2,Employees,64988,0,0,...,0.000000,0.666667,0.666667,0.0,0.0,603.816666,0.00,0.0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,5902524,Male,1972-07-22,2008-12-31,Married,2,Employees,30647,1,0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0,False
119996,5902564,Male,1978-09-30,2008-12-31,Single,0,Business-Owners,65714,0,0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0,True
119997,5902614,Female,1984-11-26,2008-12-31,Married,0,Employees,4730,0,0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0,False
119998,5902625,Male,1968-02-21,2008-12-31,Married,2,Business-Owners,30476,0,0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,1,False


In [10]:
df = df.to_pandas_dataframe()

In [12]:
df.head()

,Customer_ID,Gender,Birth_Date,Ref_Date,Marital_Status,Children_Num,Occupation_Category,Total_Income,Payroll_Flag,Business_Flag,...,Transfer_Trans_Amount,Credit_Cards_Installments,Credit_Cards_Payments_Num,Credit_Cards_Purchases_Num,Credit_Cards_Witrhdrawals_Num,Credit_Cards_Payments_Amount,Credit_Cards_Purchases_Amount,Credit_Cards_Witrhdrawals_Amount,Arrears_Months_Max,Time_Deposits_Flag
0,36163,Male,1968-01-20,2008-12-31,Single,0,Employees,17637,0,0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,1,False
1,5904434,Male,1973-04-07,2008-12-31,Single,0,Rentiers,5800,0,0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,1,False
2,5904453,Male,1960-10-17,2008-12-31,Married,2,Rentiers,14000,0,0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,1,False
3,855752,Male,1969-07-28,2008-12-31,Married,2,Business-Owners,54814,0,0,...,105.0,1.666667,0.666667,1.0,0.0,141.666666,230.45,0.0,1,False
4,81,Male,1950-07-27,2008-12-31,Married,2,Employees,64988,0,0,...,0.0,0.666667,0.666667,0.0,0.0,603.816666,0.00,0.0,1,False
